In [ ]:
!pip install opencv-python
!pip install mediapipe
!pip install pytubefix
!pip install moviepy
!pip install opencv-python-headless numpy
!pip install torch torchvision
!pip install yolov5  # if using YOLOv5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━

In [ ]:
import pandas as pd
import json

import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

from google.colab.patches import cv2_imshow

from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path_json = '/content/drive/MyDrive/PumpkinSeeds/datasource/dataLinksJson/'
file_path_vido = '/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/'
file_path_test = '/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideosTest/'
file_path_actn = '/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/'

In [ ]:
actions = set()

In [ ]:
def getDataSetAsDataFrame(file_name):
    with open(file_path_json + file_name, 'r') as file:
        json_data = json.load(file)
    return pd.DataFrame(json_data)

In [ ]:
df_train = getDataSetAsDataFrame('MSASL_train.json')
df_test = getDataSetAsDataFrame('MSASL_test.json')

In [ ]:
df_train

org_text     clean_text  start_time  signer_id  signer  \
0      match [light-a-MATCH]          match       0.000          0       0   
1                       FAIL           fail       0.000          0      -1   
2                      laugh          laugh       0.000          4      26   
3                       BOOK           book       0.000          0      -1   
4              sign-language  sign language       0.000          0      -1   
...                      ...            ...         ...        ...     ...   
16049                  WRITE          write     539.939         32      17   
16050                    hot            hot       0.000          2      42   
16051                     hi             hi       0.000          0       0   
16052                 Learn           learn       0.000        135      -1   
16053             accountant     accountant       0.000          2      81   

       start    end                          file  label  height     fps  \
0          0     83           match light-a-MATCH    830   360.0  30.000   
1          0     74                          FAIL    542   360.0  25.000   
2          0     31  SignSchool Laugh with Legs 2    312   360.0  29.970   
3          0     66                       BOOK(3)     38   360.0  25.000   
4          0     75        SIGN-LANGUAGE-S-CLAW-F    848   360.0  29.970   
...      ...    ...                           ...    ...     ...     ...   
16049  16182  16301       ASL 1 Unit 1 Vocabulary     53   360.0  29.970   
16050      0    127                           Hot    135   360.0  29.657   
16051      0     47                            Hi    379   360.0  29.970   
16052      0    104                    ASL Learn      22   360.0  30.000   
16053      0    156                    Accountant    777   360.0  29.658   

       end_time                                          url           text  \
0         2.767  https://www.youtube.com/watch?v=C37R_Ix8-qs          match   
1         2.960  https://www.youtube.com/watch?v=PIsUJl8BN_I           fail   
2         1.034          www.youtube.com/watch?v=9FdHlMOnVjg          laugh   
3         2.640  https://www.youtube.com/watch?v=J7tP98oDxqE           book   
4         2.502          www.youtube.com/watch?v=N2mG9ZKjrGA  sign language   
...         ...                                          ...            ...   
16049   543.910  https://www.youtube.com/watch?v=fNg_sJ9f8EI          write   
16050     4.282  https://www.youtube.com/watch?v=MkXUHhsMAns            hot   
16051     1.568          www.youtube.com/watch?v=rhfJGeMDMzQ             hi   
16052     3.467  https://www.youtube.com/watch?v=n-b2NMAwk28          learn   
16053     5.260  https://www.youtube.com/watch?v=YwF_2ko6RxQ     accountant   

                                                     box  width  review  
0      [0.05754461884498596, 0.21637457609176636, 1.0...  640.0     NaN  
1      [0.0657794177532196, 0.16717177629470825, 0.93...  480.0     NaN  
2      [0.13188594579696655, 0.32334136962890625, 1.0...  640.0     NaN  
3      [0.05569887161254883, 0.25173279643058777, 0.9...  480.0     NaN  
4      [0.03904399275779724, 0.24198183417320251, 1.0...  640.0     NaN  
...                                                  ...    ...     ...  
16049  [0.03217703104019165, 0.3161315321922302, 1.0,...  640.0     NaN  
16050  [0.00036713480949401855, 0.20649650692939758, ...  640.0     NaN  
16051  [0.013705313205718994, 0.17314371466636658, 1....  640.0     NaN  
16052  [0.18269836902618408, 0.19255371391773224, 1.0...  480.0     NaN  
16053  [0.0, 0.20838260650634766, 1.0, 0.855440020561...  640.0     NaN  

[16054 rows x 17 columns]

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

# Function to extract part of the video
def extract_video_segment(input_video_path, output_video_path, start_time, end_time):
    try:
        # Load the video file
        video = VideoFileClip(input_video_path)

        # Extract the part of the video between start_time and end_time
        video_subclip = video.subclip(start_time, end_time)

        # Write the result to a file
        video_subclip.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

        print(f"Video segment saved to {output_video_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
from pytubefix import YouTube
import os

def download_youtube_video(listofVideos, save_path="."):
    for index, row in listofVideos.iterrows():
        try:
            yt = YouTube(row['url'])
            temp_video_path = file_path_vido + 'temp.mp4'
            stream = yt.streams.get_highest_resolution()
            stream.download(output_path=save_path, filename=f"temp.mp4")

            print(f"Downloaded: {yt.title} successfully to {save_path}")

            output_video_path = file_path_vido + row['clean_text'] + '_' + str(index) + '.mp4'
            extract_video_segment(temp_video_path, output_video_path, row['start_time'], row['end_time'])
            actions.add(row['clean_text'])
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
documents = [
    "Hello, How are you",
    "I love my community",
    "who is cooking today",
    "It is lovely weather we are having. I hope the weather continues ",
]
# How is weather today

tokens = [word_tokenize(sentence.lower()) for sentence in documents]
tokens = [word for token_list in tokens for word in token_list]
tokens = list(set(tokens))
print(tokens)

['weather', 'love', 'today', 'hope', 'i', 'community', ',', 'the', 'you', 'is', 'it', 'who', 'lovely', 'having', 'my', 'we', '.', 'cooking', 'continues', 'how', 'hello', 'are']


In [ ]:
df_filter = df_train[df_train['clean_text'].isin(tokens)]
df_filter

org_text clean_text  start_time  signer_id  signer  start    end  \
48     How (2 ways)        how     489.182          6      -1  12166  12357   
79              YOU        you      41.408          8      20   1241   1301   
240        My, mine         my     647.080         12       6  16177  16280   
241        My, mine         my     651.200         12       6  16280  16380   
285         Weather    weather    1094.160         12       6  27354  27464   
...             ...        ...         ...        ...     ...    ...    ...   
15935         today      today       0.000          2      26      0     73   
15936         today      today       3.916          2      26    116    202   
15937         today      today       6.819          2      26    202    262   
15980           YOU        you      13.046         32      17    391    471   
15984           WHO        who      34.167         32      17   1024   1143   

                                                    file  label  height  \
48                                     LASL - Ch 6 Vocab     59   358.0   
79                       Basic ASL Vocabulary for Babies     68   360.0   
240    Mastering ASL Unit 3 Vocabulary signed by Dr W...    140   360.0   
241    Mastering ASL Unit 3 Vocabulary signed by Dr W...    140   360.0   
285    Mastering ASL Unit 3 Vocabulary signed by Dr W...    390   360.0   
...                                                  ...    ...     ...   
15935                                              Today    131   360.0   
15936                                              Today    131   360.0   
15937                                              Today    131   360.0   
15980                            ASL 1 Unit 1 Vocabulary     68   360.0   
15984                            ASL 1 Unit 1 Vocabulary     69   360.0   

          fps  end_time                                          url     text  \
48     24.870   496.862  https://www.youtube.com/watch?v=jQb9NL9_S6U      how   
79     29.970    43.410  https://www.youtube.com/watch?v=htsdwxJ-fTo      you   
240    25.000   651.200  https://www.youtube.com/watch?v=K8c-np9zNT8       my   
241    25.000   655.200  https://www.youtube.com/watch?v=K8c-np9zNT8       my   
285    25.000  1098.560  https://www.youtube.com/watch?v=K8c-np9zNT8  weather   
...       ...       ...                                          ...      ...   
15935  29.622     2.464  https://www.youtube.com/watch?v=zh0Q5PYPoas    today   
15936  29.622     6.819  https://www.youtube.com/watch?v=zh0Q5PYPoas    today   
15937  29.622     8.845  https://www.youtube.com/watch?v=zh0Q5PYPoas    today   
15980  29.970    15.716  https://www.youtube.com/watch?v=fNg_sJ9f8EI      you   
15984  29.970    38.138  https://www.youtube.com/watch?v=fNg_sJ9f8EI      who   

                                                     box  width  review  
48     [0.0200423002243042, 0.1574973464012146, 0.844...  640.0     NaN  
79     [0.10208573937416077, 0.32380735874176025, 1.0...  640.0     NaN  
240    [0.2095053493976593, 0.25815242528915405, 1.0,...  480.0     1.0  
241    [0.2095053493976593, 0.25815242528915405, 1.0,...  480.0     1.0  
285    [0.20960858464241028, 0.27273428440093994, 1.0...  480.0     1.0  
...                                                  ...    ...     ...  
15935                [0.0, 0.0, 1.0, 0.8556264042854309]  640.0     1.0  
15936                [0.0, 0.0, 1.0, 0.8556264042854309]  640.0     1.0  
15937                [0.0, 0.0, 1.0, 0.8556264042854309]  640.0     1.0  
15980  [0.02041187882423401, 0.23619934916496277, 1.0...  640.0     NaN  
15984  [0.019341468811035156, 0.2829495966434479, 1.0...  640.0     NaN  

[267 rows x 17 columns]

In [ ]:
df_filter_test = df_test[df_test['clean_text'].isin(tokens)]
df_filter_test

org_text clean_text  start_time  signer_id  signer  start  \
154               HELLO      hello       0.870         34      11     26   
257                 HOW        how      89.256         10      26   2675   
267                 WHO        who     202.469         10      26   6068   
268   WHO (alternative)        who     206.039         10      26   6175   
364                  MY         my      83.116         26       2   2491   
...                 ...        ...         ...        ...     ...    ...   
4032               WHO?        who     112.279         26       2   3365   
4038               HOW?        how     131.898         26       2   3953   
4039               HOW?        how     135.902         26       2   4073   
4040               HOW?        how     138.739         26       2   4158   
4041               HOW?        how     142.509         26       2   4271   

       end                                               file  label  height  \
154     60  ASL Unit 5 Vocabulary - Clothing Part 1 and 2 ...      0   720.0   
257   2794  Lesson 8 Vocabulary - A Basic Course (ABC) in ASL     59   720.0   
267   6168  Lesson 8 Vocabulary - A Basic Course (ABC) in ASL     69   720.0   
268   6255  Lesson 8 Vocabulary - A Basic Course (ABC) in ASL     69   720.0   
364   2537                             ASL1 Class 3 New SIGNS    140   720.0   
...    ...                                                ...    ...     ...   
4032  3416  ASL1 Class 4 New SIGNS plus Partner Practice Q...     69   720.0   
4038  4013  ASL1 Class 4 New SIGNS plus Partner Practice Q...     59   720.0   
4039  4143  ASL1 Class 4 New SIGNS plus Partner Practice Q...     59   720.0   
4040  4224  ASL1 Class 4 New SIGNS plus Partner Practice Q...     59   720.0   
4041  4313  ASL1 Class 4 New SIGNS plus Partner Practice Q...     59   720.0   

         fps  end_time                                          url   text  \
154   29.892     2.007  https://www.youtube.com/watch?v=-DMFu-nAsok  hello   
257   29.970    93.226  https://www.youtube.com/watch?v=HuViTIjghKk    how   
267   29.970   205.806  https://www.youtube.com/watch?v=HuViTIjghKk    who   
268   29.970   208.709  https://www.youtube.com/watch?v=HuViTIjghKk    who   
364   29.970    84.651  https://www.youtube.com/watch?v=KTURNRi-wy0     my   
...      ...       ...                                          ...    ...   
4032  29.970   113.981  https://www.youtube.com/watch?v=RMUydgQr9TE    who   
4038  29.970   133.900  https://www.youtube.com/watch?v=RMUydgQr9TE    how   
4039  29.970   138.238  https://www.youtube.com/watch?v=RMUydgQr9TE    how   
4040  29.970   140.941  https://www.youtube.com/watch?v=RMUydgQr9TE    how   
4041  29.970   143.910  https://www.youtube.com/watch?v=RMUydgQr9TE    how   

                                                    box   width  review  
154   [0.004950165748596191, 0.0, 1.0, 0.78116858005...  1280.0     NaN  
257   [0.30637162923812866, 0.28579655289649963, 0.9...  1280.0     NaN  
267   [0.2714803218841553, 0.33794939517974854, 0.97...  1280.0     NaN  
268   [0.2680490016937256, 0.31983017921447754, 0.97...  1280.0     NaN  
364   [0.13907408714294434, 0.17668381333351135, 1.0...  1280.0     NaN  
...                                                 ...     ...     ...  
4032  [0.17970159649848938, 0.19653379917144775, 1.0...  1280.0     NaN  
4038  [0.16711995005607605, 0.10408169031143188, 1.0...  1280.0     1.0  
4039  [0.16711995005607605, 0.10408169031143188, 1.0...  1280.0     1.0  
4040  [0.16711995005607605, 0.10408169031143188, 1.0...  1280.0     1.0  
4041  [0.17888954281806946, 0.13472670316696167, 0.9...  1280.0     NaN  

[61 rows x 17 columns]

In [ ]:
download_youtube_video(df_filter, save_path=file_path_vido)

Downloaded: LASL - Ch. 6 Vocab successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_48.mp4.
MoviePy - Writing audio in how_48TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_48.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_48.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_48.mp4
Downloaded: Basic ASL Vocabulary for Babies successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_79.mp4.
MoviePy - Writing audio in you_79TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_79.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_79.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_79.mp4
Downloaded: Mastering ASL Unit 3 Vocabulary signed by Dr. Wooten successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_240.mp4.
MoviePy - Writing audio in my_240TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/today_15937.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/today_15937.mp4
Downloaded: ASL 1: Unit 1 Vocabulary successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_15980.mp4.
MoviePy - Writing audio in you_15980TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_15980.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_15980.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_15980.mp4
Downloaded: ASL 1: Unit 1 Vocabulary successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/who_15984.mp4.
MoviePy - Writing audio in who_15984TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/who_15984.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/who_15984.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/who_15984.mp4


In [ ]:
download_youtube_video(df_filter_test, save_path=file_path_vido)

Downloaded: ASL Unit 5 Vocabulary - Clothing Part 1 and 2. Turn on [CC] successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/hello_154.mp4.
MoviePy - Writing audio in hello_154TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/hello_154.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/hello_154.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/hello_154.mp4
An error occurred: HuViTIjghKk is unavailable
An error occurred: HuViTIjghKk is unavailable
An error occurred: HuViTIjghKk is unavailable
Downloaded: ASL1 Class 3 New SIGNS successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_364.mp4.
MoviePy - Writing audio in my_364TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_364.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_364.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_364.mp4
Downloaded: ASL2 Class 52 SIGNS successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/hello_491.mp4.
MoviePy - Writing audio in hello_491TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4039.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4039.mp4
Downloaded: ASL1 Class 4 New SIGNS plus Partner Practice Q&A and p29 Dialogue successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4040.mp4.
MoviePy - Writing audio in how_4040TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4040.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4040.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4040.mp4
Downloaded: ASL1 Class 4 New SIGNS plus Partner Practice Q&A and p29 Dialogue successfully to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/
Moviepy - Building video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4041.mp4.
MoviePy - Writing audio in how_4041TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4041.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4041.mp4
Video segment saved to /content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_4041.mp4


In [ ]:
# Function to find the maximum number of frames across all videos
def find_max_frames(video_folder):
    min_frames = 9999999
    max_frames = 0
    for video_file in os.listdir(video_folder):
        if video_file.endswith('.mp4') and video_file != 'temp.mp4':
            video_path = os.path.join(video_folder, video_file)
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            min_frames = min(min_frames, frame_count)
            max_frames = max(max_frames, frame_count)
            cap.release()
    return min_frames, max_frames

In [ ]:
train_frames_min, train_frames_max = find_max_frames(file_path_vido)
test_frames_min,  test_frames_max  = find_max_frames(file_path_test)

print(f"Min, Maxi number of frames found on Train: {train_frames_min}, {train_frames_max}")
print(f"Min, Maxi number of frames found on Test : {test_frames_min},  {test_frames_max}")

Min, Maxi number of frames found on Train: 16, 200
Min, Maxi number of frames found on Test : 19,  90


# Feature Extraction

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
'''
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
'''

'\ndef draw_landmarks(image, results):\n    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections\n    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections\n    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections\n    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections\n'

In [ ]:
def draw_landmarks(image, results):
    # Draw face mesh connections
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=results.face_landmarks,
        connections=mp.solutions.face_mesh_connections.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(80,110,10), thickness=1)
    )

    # Draw pose connections
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=results.pose_landmarks,
        connections=mp.solutions.pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(80,44,121), thickness=2)
    )

    # Draw left hand connections
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=results.left_hand_landmarks,
        connections=mp.solutions.hands.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(121,44,250), thickness=2)
    )

    # Draw right hand connections
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=results.right_hand_landmarks,
        connections=mp.solutions.hands.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(245,66,230), thickness=2)
    )

In [ ]:
'''
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )
'''

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
def extract_features_from_video(input_video_path):

    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    idx = np.round(np.linspace(0, frames - 1, train_frames_min)).astype(int)
    print(input_video_path)
    print(idx)
    '''
    if frames > 42:
        return
    '''
    # Set mediapipe model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        action = input_video_path[input_video_path.rfind('/')+1:]
        action = action[:action.find('_')]

        sequence = input_video_path[input_video_path.rfind('_')+1:]
        sequence = sequence[:sequence.find('.')]

        frame_index = 0
        frame_num = 0
        while cap.isOpened():
            if frame_num >= train_frames_min:
                break
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx[frame_num])
            ret, frame = cap.read()
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)
            #print(results)

            # Draw landmarks
            draw_landmarks(image, results)

            # NEW Export keypoints
            keypoints = extract_keypoints(results)
            npy_path = file_path_actn + action + '/' + str(sequence)
            Path(npy_path).mkdir(parents=True, exist_ok=True)

            npy_path += '/' + str(frame_num)
            frame_num += 1

            Path(npy_path).mkdir(parents=True, exist_ok=True)
            np.save(npy_path, keypoints)

            frame_index += 1

        cap.release()
        cv2.destroyAllWindows()

In [ ]:
def extract_features_from_video_in_folder(video_folder):
    for video_file in os.listdir(video_folder):
        if video_file.endswith('.mp4') and video_file != 'temp.mp4':
            video_path = os.path.join(video_folder, video_file)
            extract_features_from_video(video_path)

In [ ]:
extract_features_from_video_in_folder(file_path_vido)

/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/you_79.mp4
[ 0  1  3  4  6  7  9 10 12 13 14 16 17 19 20 22 23 24 26 27 29 30 32 33
 35 36 37 39 40 42 43 45 46 47 49 50 52 53 55 56 58 59]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_48.mp4
[  0   5   9  14  19  23  28  33  37  42  47  52  56  61  66  70  75  80
  84  89  94  98 103 108 112 117 122 126 131 136 140 145 150 155 159 164
 169 173 178 183 187 192]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_240.mp4
[  0   3   5   8  10  13  15  18  20  23  25  28  30  33  35  38  40  43
  45  48  50  53  55  58  60  63  65  68  70  73  75  78  80  83  85  88
  90  93  95  98 100 103]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/my_241.mp4
[ 0  2  5  7 10 12 14 17 19 22 24 27 29 31 34 36 39 41 43 46 48 51 53 56
 58 60 63 65 68 70 72 75 77 80 82 85 87 89 92 94 97 99]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/weather_285.mp4
[  0   3   5   8  11  13  16

  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '



/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/love_987.mp4
[ 0  2  4  5  7  9 11 12 14 16 18 20 21 23 25 27 28 30 32 34 36 37 39 41
 43 45 46 48 50 52 53 55 57 59 61 62 64 66 68 69 71 73]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/today_994.mp4
[  0   4   9  13  17  22  26  31  35  39  44  48  52  57  61  65  70  74
  79  83  87  92  96 100 105 109 114 118 122 127 131 135 140 144 148 153
 157 162 166 170 175 179]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_1568.mp4
[  0   4   8  13  17  21  25  29  33  38  42  46  50  54  58  63  67  71
  75  79  83  88  92  96 100 104 108 113 117 121 125 129 133 138 142 146
 150 154 158 163 167 171]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_1569.mp4
[ 0  1  1  2  3  4  4  5  6  6  7  8  8  9 10 11 11 12 13 13 14 15 16 16
 17 18 18 19 20 21 21 22 23 23 24 25 25 26 27 28 28 29]
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignVideos/how_1570.mp4
[ 0  1  3  4  5  7  8 

# Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
'''
actions = [
           # 'community', 'is', ',', 'it', '.',
           # 'having', 'lovely', 'hope', 'the',
           # 'are', 'continues',

           'cooking', 'hello', 'how', 'i', 'love', 'my', 'today', 'we', 'weather', 'who', 'you']
'''

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'how': 0,
 'love': 1,
 'today': 2,
 'cooking': 3,
 'my': 4,
 'you': 5,
 'i': 6,
 'who': 7,
 'hello': 8,
 'weather': 9,
 'we': 10}

In [ ]:
sequences, labels = [], []
for action in actions:
    dir_action = file_path_actn + action
    print(dir_action)
    for video_file in os.listdir(dir_action):
        window = []
        for frame_file in os.listdir(dir_action + '/' + video_file):
            if frame_file.endswith('.npy') != True:
                continue
            final_file_name = file_path_actn + action + '/' + video_file + '/' + frame_file
            res = np.load(final_file_name)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/how
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/love
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/today
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/cooking
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/my
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/you
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/i
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/who
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/hello
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/weather
/content/drive/MyDrive/PumpkinSeeds/datasource/dataSignActions/we


In [ ]:
for sequence in sequences:
    print('Sequence Shape', np.array(sequence).shape)

Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Shape (42, 1662)
Sequence Sha

In [ ]:
np.array(sequences).shape

(227, 42, 1662)

In [ ]:
np.array(labels).shape

(227,)

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

(227, 42, 1662)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

(12, 11)

# Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 176ms/step - categorical_accuracy: 0.0876 - loss: 15.9969
Epoch 2/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - categorical_accuracy: 0.1337 - loss: 183.6674
Epoch 3/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - categorical_accuracy: 0.1480 - loss: 371.7010
Epoch 4/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 273ms/step - categorical_accuracy: 0.0986 - loss: 269.6672
Epoch 5/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 286ms/step - categorical_accuracy: 0.0967 - loss: 239.7422
Epoch 6/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 295ms/step - categorical_accuracy: 0.0960 - loss: 276.3260
Epoch 7/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - categorical_accuracy: 0.0739 - loss: 158.0136
Epoch 8/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - categorical_accuracy: 0.1045 - loss: 144.6040
Epoch 9/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - categorical_accuracy: 0.1280 - loss: 282.8939
Epoch 10/2000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - categorical_accuracy: 0.0540 - loss: 228.3704

In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 64)              │         442,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 11)                  │             363 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,790,819 (6.83 MB)

 Trainable params: 596,939 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,880 (4.55 MB)

# Save Model

In [ ]:
model.save(file_path_vido + 'action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
X_test.shape

(12, 42, 1662)

In [ ]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 0, 10],
        [ 0,  2]],

       [[10,  0],
        [ 2,  0]],

       [[11,  0],
        [ 1,  0]],

       [[10,  0],
        [ 2,  0]],

       [[10,  0],
        [ 2,  0]],

       [[ 9,  0],
        [ 3,  0]]])

In [ ]:
accuracy_score(ytrue, yhat)

0.16666666666666666